# 👂 Few shot text classification with active learning using small-text and SetFit

In this tutorial, you will learn how to set up a complete active learning loop with a [Hugging Face transformer](https://huggingface.co/docs/transformers/index):

 - Use the excellent [small-text](https://github.com/webis-de/small-text) library to set up your active learner;
 - Use a [Argilla listener](/../reference/python/python_listeners.html#python-listeners) to build and start an active learning loop;
 - Use the [Argilla UI](/../reference/webapp/features.html#annotate) to annotate examples and learn actively;

![training-textclassification-smalltext-activelearning](/_static/tutorials/training-textclassification-smalltext-activelearning/training-textclassification-smalltext-activelearning.png)

## Introduction

> Active learning is a special case of machine learning in which a learning algorithm can interactively query a user (or some other information source) to label new data points with the desired outputs. [Wikipedia](https://en.wikipedia.org/wiki/Active_learning_(machine_learning))

Supervised machine learning often requires large amounts of labeled data that are expensive to generate. 
*Active Learning* (AL) systems attempt to overcome this labeling bottleneck. 
The underlying idea is that not all data points are equally important for training the model. 
The AL system tries to query only the most relevant data from a pool of unlabeled data to be labeled by a so-called *oracle*, which is often a human annotator.
Therefore, AL systems are usually much more sample-efficient and need far less training data than traditional supervised systems.

This tutorial will show you how to incorporate [Argilla](https://github.com/argilla-io/argilla) into an active learning workflow involving a human in the loop.
We will build a simple text classifier by combining the active learning framework [small-text](https://github.com/webis-de/small-text) and Argilla. 
Hugging Face's [transformers](https://github.com/huggingface/transformers) will provide the classifier we will embed in an active learner from small-text. Argilla excels in making **you** the oracle that conveniently teaches the model via an intuitive UI.

## Running Argilla

For this tutorial, you will need to have an Argilla server running. There are two main options for deploying and running Argilla:


**Deploy Argilla on Hugging Face Spaces**: If you want to run tutorials with external notebooks (e.g., Google Colab) and you have an account on Hugging Face, you can deploy Argilla on Spaces with a few clicks:

[![deploy on spaces](https://huggingface.co/datasets/huggingface/badges/raw/main/deploy-to-spaces-lg.svg)](https://huggingface.co/new-space?template=argilla/argilla-template-space)

For details about configuring your deployment, check the [official Hugging Face Hub guide](https://huggingface.co/docs/hub/spaces-sdks-docker-argilla).


**Launch Argilla using Argilla's quickstart Docker image**: This is the recommended option if you want [Argilla running on your local machine](/../getting_started/quickstart_installation.html). Note that this option will only let you run the tutorial locally and not with an external notebook service.

For more information on deployment options, please check the Deployment section of the documentation.

<div class="alert alert-info">

Tip
    
This tutorial is a Jupyter Notebook. There are two options to run it:

- Use the Open in Colab button at the top of this page. This option allows you to run the notebook directly on Google Colab. Don't forget to change the runtime type to GPU for faster model training and inference.
- Download the .ipynb file by clicking on the View source link at the top of the page. This option allows you to download the notebook and run it on your local machine or on a Jupyter Notebook tool of your choice.
</div>

In [ ]:
%pip install "argilla[listeners]" "datasets~=2.5.0" "small-text>=1.1.0"  "transformers[torch]"

Let's import the Argilla module for reading and writing data:

In [ ]:
import argilla as rg

If you are running Argilla using the Docker quickstart image or Hugging Face Spaces, you need to init the Argilla client with the `URL` and `API_KEY`:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
# Replace workspace with the name of your workspace
rg.init(
    api_url="http://localhost:6900", 
    api_key="owner.apikey",
    workspace="admin"
)

If you're running a private Hugging Face Space, you will also need to set the [HF_TOKEN](https://huggingface.co/settings/tokens) as follows:

In [ ]:
# # Set the HF_TOKEN environment variable
# import os
# os.environ['HF_TOKEN'] = "your-hf-token"

# # Replace api_url with the url to your HF Spaces URL
# # Replace api_key if you configured a custom API key
# # Replace workspace with the name of your workspace
# rg.init(
#     api_url="https://[your-owner-name]-[your_space_name].hf.space", 
#     api_key="owner.apikey",
#     workspace="admin",
#     extra_headers={"Authorization": f"Bearer {os.environ['HF_TOKEN']}"},
# )

### Enable Telemetry

We gain valuable insights from how you interact with our tutorials. To improve ourselves in offering you the most suitable content, using the following lines of code will help us understand that this tutorial is serving you effectively. Though this is entirely anonymous, you can choose to skip this step if you prefer. For more info, please check out the [Telemetry](../../reference/telemetry.md) page.

In [ ]:
try:
    from argilla.utils.telemetry import tutorial_running
    tutorial_running()
except ImportError:
    print("Telemetry is introduced in Argilla 1.20.0 and not found in the current installation. Skipping telemetry.")

## The AG's News dataset

We use the "AG's News" dataset for this demonstration. It consists of news articles that are categorized into four mutually exclusive classes.

Let us load the dataset from the [Hugging Face Hub](https://huggingface.co/datasets):

In [ ]:
import datasets
import logging
import numpy as np


# Disables the progress bar for notebooks: https://github.com/huggingface/datasets/issues/2651
datasets.logging.get_verbosity = lambda: logging.NOTSET

raw_dataset = datasets.load_dataset('ag_news')
num_classes = np.unique(raw_dataset['train']['label']).shape[0]

print('First 10 training samples:\n')
for i in range(10):
    print(raw_dataset['train']['label'][i], ' ', raw_dataset['train']['text'][i])

## Preprocessing the dataset

The SetFit Classifier uses the TextData structure as input. Such datasets can be easily created from raw text and labels using TextDataset.from_arrays().

In [ ]:
import numpy as np
from small_text import TextDataset

num_classes = raw_dataset['train'].features['label'].num_classes

target_labels = np.arange(num_classes)

train = TextDataset.from_arrays(
    raw_dataset['train']['text'], 
    np.array(raw_dataset['train']['label']), 
    target_labels=target_labels
)
test = TextDataset.from_arrays(
    raw_dataset['test']['text'], 
    np.array(raw_dataset['test']['label']), 
    target_labels=target_labels
)

## Setting up the active learner

Now that we have our datasets ready let's set up the active learner. 
For this, we need two components:

 - the **classifier** to be trained;
 - the **query strategy** to obtain the most relevant data;
 
In our case, we choose a [Hugging Face transformer](https://huggingface.co/docs/transformers/index) as the classifier and a [tie-breaker](https://small-text.readthedocs.io/en/v1.0.0/components/query_strategies.html#small_text.query_strategies.strategies.BreakingTies) as the query strategy. 
The latter selects instances of the data pool with a small margin between the two most likely predicted labels. 

In [ ]:
import logging

from small_text import (
    BreakingTies,
    PoolBasedActiveLearner,
    SetFitClassificationFactory,
    SetFitModelArguments,
)

# Set logger to get training feedback
logging.getLogger("small_text").setLevel(logging.INFO)

# Define our classifier
sentence_transformer_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
setfit_model_args = SetFitModelArguments(sentence_transformer_model_name)
clf_factory = SetFitClassificationFactory(
    setfit_model_args, 
    num_classes
)

# Define our query strategy
query_strategy = BreakingTies()
setfit_train_kwargs = {'show_progress_bar': False}

# Use the active learner with a pool containing all unlabeled data
active_learner = PoolBasedActiveLearner(
    clf_factory, 
    query_strategy, 
    train, 
    fit_kwargs={'setfit_train_kwargs': setfit_train_kwargs}
)

Since most query strategies, including ours, require a trained model, we randomly draw a subset from the data pool to initialize our AL system. 
After obtaining the labels for this batch of instances, the active learner will use them to create the first classifier.


In [ ]:
from small_text import random_initialization
import numpy as np

# Fix seed for reproducibility
np.random.seed(42)

# Number of samples in our queried batches
NUM_SAMPLES = 20

# Randomly draw an initial subset from the data pool
initial_indices = random_initialization(dataset, NUM_SAMPLES)


## Argilla and you: the perfect oracle

With our active learner ready, it is time to teach it. 
We first create a [Argilla dataset](/../practical_guides/create_update_dataset/create_dataset.html) to log and label the initial random batch queried by the active learner.

In [ ]:
import argilla as rg

# Choose a name for the dataset
DATASET_NAME = "ag_with_active_learning"

# Define the labeling schema
labels = raw_dataset["train"].features["label"].names
settings = rg.TextClassificationSettings(label_schema=labels)

# Create a dataset with a label schema
rg.configure_dataset_settings(name=DATASET_NAME, settings=settings)

# Create records from the initial batch
records = [
    rg.TextClassificationRecord(
        text=raw_dataset["train"]["text"][idx],
        metadata={"batch_id": 0},
        id=int(idx),
    )
    for idx in initial_indices
]

# Log the initial records to Argilla
rg.log(records, DATASET_NAME)


Before switching to the Argilla UI to label the records, we will set up the **active learning loop**. 
For this, we will use the [listener decorator](/reference/python/python_listeners.html#argilla.listeners.listener) from Argilla.
The loop will run automatically once all records of a batch are labeled (see the `query` and `condition` argument of the decorator). 
It will trigger the classifier's training, query a new batch from the active learner, and log it to Argilla. 
We will also keep track of the accuracy of the current classifier by evaluating it on our test set.

In [ ]:
from argilla.listeners import listener
from sklearn.metrics import accuracy_score

# Define some helper variables
LABEL2INT = raw_dataset["train"].features["label"].str2int
ACCURACIES = []

# Set up the active learning loop with the listener decorator
@listener(
    dataset=DATASET_NAME,
    query="status:Validated AND metadata.batch_id:{batch_id}",
    condition=lambda search: search.total == NUM_SAMPLES,
    execution_interval_in_seconds=3,
    batch_id=0,
)
def active_learning_loop(records, ctx):
    # 1. Update active learner
    print(f"Updating with batch_id {ctx.query_params['batch_id']} ...")
    y = np.array([LABEL2INT(rec.annotation) for rec in records])

    # initial update
    print(
        "Starting training! Depending on your device, this might take a while, so set up"
        " logging."
    )
    if ctx.query_params["batch_id"] == 0:
        indices = np.array([rec.id for rec in records])
        active_learner.initialize_data(indices, y)
    # update with the prior queried indices
    else:
        active_learner.update(y)
    print("Done!")

    # 2. Query active learner
    print("Querying new data points ...")
    queried_indices = active_learner.query(num_samples=NUM_SAMPLES)
    new_batch = ctx.query_params["batch_id"] + 1
    new_records = [
        rg.TextClassificationRecord(
            text=trec["train"]["text"][idx],
            metadata={"batch_id": new_batch},
            id=idx,
        )
        for idx in queried_indices
    ]

    # 3. Log the batch to Argilla
    rg.log(new_records, DATASET_NAME)

    # 4. Evaluate the current classifier on the test set
    print("Evaluating current classifier ...")
    accuracy = accuracy_score(
        dataset_test.y,
        active_learner.classifier.predict(dataset_test),
    )

    ACCURACIES.append(accuracy)
    ctx.query_params["batch_id"] = new_batch
    print("Done!")

    print("Waiting for annotations ...")


## Starting the active learning loop

Now we can start the loop and switch to the Argilla UI.

In [ ]:
active_learning_loop.start()


In the Argilla UI, we will set the number of records per page to 20 since it is also our chosen batch size. 
Furthermore, we will use the [Status filter](/../reference/webapp/pages.html#filters) to filter out already annotated records. 
Now, all we have to do is to annotate the displayed records. 
Once annotating everything, the classifier's training will be automatically triggered.

After a few seconds, you should see the newly queried batch when pressing the [Refresh button](/../reference/webapp/pages.html#refresh). 
The training can take longer depending on your machine and whether you have a CUDA device. 
You can always check the status of the active learning loop from your notebook.

## Can we stop?

After a few iterations, we can check the accuracy of the current classifier and plot its evaluation.

In [ ]:
import pandas as pd

pd.Series(ACCURACIES).plot(xlabel="Iteration", ylabel="Accuracy")


We should achieve an accuracy of at least **0.8 after around 12 iterations**, corresponding to roughly 260 annotated records. 
The stopping criterion is ultimately up to you, and you can choose more sophisticated criteria like the [KappaAverage](https://small-text.readthedocs.io/en/v1.0.0/components/stopping_criteria.html) implemented in small-text.

In [ ]:
active_learning_loop.stop()


## Summary

In this tutorial, we saw how you could **embed Argilla in an active learning loop involving a human in the loop**. 
We relied on **small-text to use a Hugging Face transformer within an active learning setup**. 
In the end, we gathered **a sample-efficient data set by annotating only the most informative records** for the model.

Argilla makes it very easy to use a dedicated annotation team or subject matter experts as an oracle for your active learning system. They will only interact with the Argilla UI and do not have to worry about training or querying the system. We encourage you to try out active learning in your next project and make your and your annotator's life a little easier.